<a href="https://colab.research.google.com/github/msommerfe/CAS-Applied-Data-Science/blob/master/Module6_Deep_Learning/Object_DetectionV8_GoogleColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install ultralytics

In [3]:
import ssl
import urllib
import os
import time
from matplotlib import pyplot as plt
import numpy as np
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import plot_results
print(os.getcwd())

/content


In [4]:
!nvidia-smi

Fri Mar  8 15:02:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              41W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
#!python train.py --img 1280 --batch 16 --epochs 500 --data /content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1200x900/YoloTrainingPath.yaml --weights yolov5s.pt
yamlPath = '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/YoloTrainingPath.yaml'
model = YOLO('yolov8n.pt')

results = model.train(data = yamlPath, epochs = 500, imgsz=1280)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!cp -av "/content/runs" "/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960"

In [ ]:
resultPath = '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/runs/detect/train2/results'
#plot_results(resultPath + '.csv') #Creating a image (result.png) based on data in result.csv

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(resultPath + '.png')
plt.imshow(img)
plt.show()

In [4]:
pathBestWeights = '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/runs/detect/train/weights/best.pt'
model = YOLO(pathBestWeights)

In [ ]:
img = '/content/drive/MyDrive/Colab_Notebooks/Data/Try_EVN_1280x960/validate/images'
results = model(img)

In [ ]:
#super sad, cv2.imshow is not working in colab. Thats why workaround is needed
#for r in results:
#        cv2.imshow('YOLO', np.squeeze(r.plot()))

for r in results:
  plt.imshow(np.squeeze(r.plot()))
  plt.show()

In [20]:
videoPath = '/content/drive/MyDrive/Colab_Notebooks/Data/Video/480p.mp4'
#results = model(source = videoPath, save = True, stream = True)

In [ ]:
model.predict('/content/drive/MyDrive/Colab_Notebooks/Data/Video/full.mp4', save=True, imgsz=1280, augment = True, conf = 0.1)

In [ ]:
!ffmpeg -i {"runs/detect/predict/full.avi"} -vcodec libx264 {"final2.mp4"}

In [22]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('final2.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Neuer Abschnitt